In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cap = cv2.VideoCapture("/content/gdrive/MyDrive/m2.mp4")

영상 추출 관련 기본 데이터

In [4]:
fps = round(cap.get(cv2.CAP_PROP_FPS))
t = 30
end_sec = 110
interval = 5
# 단위는 초
print(f"fps : {fps}")

fps : 60


In [5]:
up_pos = np.array([[1265,914],[1274,923]])
down_pos = np.array([[1265,992],[1274,1001]])
left_pos = np.array([[1187,992],[1206,1001]])
right_pos = np.array([[1343,992],[1352,1001]])
ctrl_pos = np.array([[1091,992],[1110,1001]])
shift_pos = np.array([[1091,914],[1110,923]])

In [6]:
while t <= end_sec:
  cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
  ret,frame = cap.read()

  if not ret:
    print("영상 시간 초과 또는 영상 관련 오류")

  up_pushed = 1 if (np.sum(frame[up_pos[0,1]:up_pos[1,1],up_pos[0,0]:up_pos[1,0]]) < 50000) else 0
  down_pushed = 1 if (np.sum(frame[down_pos[0,1]:down_pos[1,1],down_pos[0,0]:down_pos[1,0]]) < 50000) else 0
  left_pushed = 1 if (np.sum(frame[left_pos[0,1]:left_pos[1,1],left_pos[0,0]:left_pos[1,0]]) < 50000) else 0
  right_pushed = 1 if (np.sum(frame[right_pos[0,1]:right_pos[1,1],right_pos[0,0]:right_pos[1,0]]) < 50000) else 0
  ctrl_pushed = 1 if (np.sum(frame[ctrl_pos[0,1]:ctrl_pos[1,1],ctrl_pos[0,0]:ctrl_pos[1,0]]) < 50000) else 0
  shift_pushed = 1 if (np.sum(frame[shift_pos[0,1]:shift_pos[1,1],shift_pos[0,0]:shift_pos[1,0]]) < 50000) else 0

  key_pushed = [up_pushed,left_pushed,right_pushed,ctrl_pushed,shift_pushed,down_pushed]
  keystring = ''.join(map(str,key_pushed))
  print(f"key string at {t} is {keystring}")
  cropped_img = frame[:1080,:1440]
  cv2.imwrite(f"/content/gdrive/MyDrive/kart_cap/kart_image{t}.jpg", cropped_img)
  # 키 입력 추출
  # csv 형식으로 저장 (해당 화면 이미지 파일, 다음 키 입력)
  # 이미지 파일에서 키 뷰어부분 마스킹
  t += interval

key string at 30 is 000000
key string at 35 is 000000
key string at 40 is 000000
key string at 45 is 000000
key string at 50 is 110000
key string at 55 is 110000
key string at 60 is 100000
key string at 65 is 110000
key string at 70 is 101000
key string at 75 is 101000
key string at 80 is 100000
key string at 85 is 100000
key string at 90 is 100000
key string at 95 is 101000
key string at 100 is 101000
key string at 105 is 101000
key string at 110 is 101000


In [7]:
cap.release()